In [10]:
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.conf import SparkConf

spark = SparkSession.builder.config(conf=SparkConf()).master("local").appName("RankTimeSeries").getOrCreate()

# 11809233 


In [11]:
from pyspark.sql.functions import unix_timestamp,rank, desc
from pyspark.sql.window import Window

year = 2015

df = spark.read.format("csv").option("header", "true").load('data/Parking_Violations_Issued_-_Fiscal_Year_{}.csv'.format(year))

Dates = df.select('Summons Number', unix_timestamp('Issue Date', 'MM/dd/yyyy').alias('date')).fillna(0)
oldColumns = Dates.schema.names
newColumns = ["SN", "date"]
Dates = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), Dates)
#Dates.printSchema()
#Dates.show()
Dates.createOrReplaceTempView("initial")
Ranked = spark.sql("""
    SELECT i.SN, i.date, ROW_NUMBER() OVER (ORDER BY i.date DESC) AS position
    FROM initial as i
    """)
#Ranked = Dates.withColumn("position", rank().over(window))
#Ranked.coalesce(1)

#Ranked.write.format('com.databricks.spark.csv').save('data/ranked',mode='overwrite')



In [12]:
Ranked.show()

+----------+----------+--------+
|        SN|      date|position|
+----------+----------+--------+
|7713884014|1435622400|       1|
|1383240346|1435622400|       2|
|8027900840|1435622400|       3|
|5087037630|1435622400|       4|
|7575453805|1435622400|       5|
|5087045698|1435622400|       6|
|7099890669|1435622400|       7|
|4004752449|1435622400|       8|
|7249051659|1435622400|       9|
|5087040494|1435622400|      10|
|8043760111|1435622400|      11|
|1385767900|1435622400|      12|
|1384524812|1435622400|      13|
|5087044463|1435622400|      14|
|8025982324|1435622400|      15|
|1379731926|1435622400|      16|
|7582369506|1435622400|      17|
|1385282368|1435622400|      18|
|7298797645|1435622400|      19|
|4004751524|1435622400|      20|
+----------+----------+--------+
only showing top 20 rows



In [14]:
import os

X = Ranked.toPandas() 

Py4JJavaError: An error occurred while calling o150.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 140, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:260)
	at org.apache.spark.scheduler.DirectTaskResult$$anonfun$writeExternal$1.apply$mcV$sp(TaskResult.scala:50)
	at org.

In [ ]:
X

In [ ]:
df = spark.read.format("csv").option("header", "true").load('data/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')

Dates = df.select('Summons Number')
window = Window.orderBy(desc('Summons Number'))
Ranked = Dates.withColumn("position", rank().over(window))
print(Ranked.count())

